Orodja

In [4]:
import csv
import os
import requests
import sys
import re


def pripravi_imenik(ime_datoteke):
    '''Če še ne obstaja, pripravi prazen imenik za dano datoteko.'''
    imenik = os.path.dirname(ime_datoteke)
    if imenik:
        os.makedirs(imenik, exist_ok=True)


def shrani(url, ime_datoteke, vsili_prenos=False):
    '''Vsebino strani na danem naslovu shrani v datoteko z danim imenom.'''
    try:
        print('Shranjujem {}...'.format(url), end='')
        sys.stdout.flush()
        if os.path.isfile(ime_datoteke) and not vsili_prenos:
            print('shranjeno že od prej!')
            return
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        print('stran ne obstaja!')
    else:
        pripravi_imenik(ime_datoteke)
        with open(ime_datoteke, 'w') as datoteka:
            datoteka.write(r.text)
            print('shranjeno!')


def vsebina_datoteke(ime_datoteke):
    '''Vrne niz z vsebino datoteke z danim imenom.'''
    with open(ime_datoteke) as datoteka:
        vsebina = datoteka.read()
    return vsebina


def datoteke(imenik):
    '''Vrne imena vseh datotek v danem imeniku skupaj z imenom imenika.'''
    return [os.path.join(imenik, datoteka) for datoteka in os.listdir(imenik)]


def zapisi_tabelo(slovarji, imena_polj, ime_datoteke):
    '''Iz seznama slovarjev ustvari CSV datoteko z glavo.'''
    pripravi_imenik(ime_datoteke)
    with open(ime_datoteke, 'w') as csv_dat:
        writer = csv.DictWriter(csv_dat, fieldnames=imena_polj)
        writer.writeheader()
        for slovar in slovarji:
            writer.writerow(slovar)

Funkcija, ki z regularnimi izrazi izlušči podatke iz html kode

In [36]:
regex_db = re.compile(
        r'<a href="(?P<povezava>/athlete/\d+)">\s*(?P<ime>.*?\s.*?)\s*</a>'  
        r'.*?<img src="https:.*?<small>(?P<drzava>.*?)</small>'
        r'.*?<td class="text-center">\s*(?P<tocke>\d+) points\s*</td>'
        ,flags=re.DOTALL
)

def pocisti_ime(judoka):
    podatki = judoka.groupdict()
    podatki['ime'].capitalize()
    podatki['ime'] = re.sub('\s+',' ',podatki['ime'])
        
    return podatki

def pripravi_db():   
    judoke = []
    for dat in datoteke('JUDO/'):
        
        for judoka in re.finditer(regex_db, vsebina_datoteke(dat)):

            #print (pocisti_ime(judoka))
            judoke.append(pocisti_ime(judoka))

    zapisi_tabelo(judoke, ['povezava','ime', 'drzava', 'tocke'], 'judoke.csv')

    
pripravi_db()